In [18]:
import os
import sys
import re
from shutil import rmtree
import base64
from github import Github, GithubException, UnknownObjectException

In [19]:
ORG = 'Nino-cunei'
REPO = 'oldbabylonian'
COMMIT = '20173f788d445e60e4bc40891f2fee26044119a1'
DWN = os.path.expanduser('~/Downloads')

In [20]:
g = Github()
repoObj = g.get_repo(f'{ORG}/{REPO}')

In [26]:
repoObj.get_archive_link('zipball')

'https://codeload.github.com/Nino-cunei/oldbabylonian/legacy.zip/master'

In [9]:
repoObj2 = g.get_repo(f'Dans-labs/ehri-tools')

In [16]:
try:
  releaseObjET = repoObj2.get_release('v1.4')
except UnknownObjectException:
  print('No release tagged "v1.4"')
except Exception as exc:
  print(f'error {exc}')

No release tagged "v1.4"


In [17]:
try:
  releaseObjET = repoObj2.get_latest_release()
except UnknownObjectException:
  print('No releases')
except Exception as exc:
  print(f'error {exc}')

No releases


In [4]:
releaseObj = repoObj.get_latest_release()

In [5]:
releaseObj

GitRelease(title="New data version")

In [6]:
releaseObj2 = repoObj.get_release('v1.4')

In [8]:
releaseObj == releaseObj2

True

In [14]:
releaseObj.tag_name

'v1.4'

In [7]:
releaseObj2.tag_name

'v1.4'

In [16]:
releaseObj.target_commitish

'master'

In [18]:
ref = repoObj.get_git_ref(f'tags/{releaseObj.tag_name}')

In [19]:
ref

GitRef(ref="refs/tags/v1.4")

In [21]:
ref.object.sha

'43c36d148794e3feeb3dd39e105ce6a4df79c467'

In [6]:
commitsObj = repoObj.get_commits()

In [9]:
commitsObj.totalCount

41

In [10]:
commitsObj[0]

Commit(sha="20173f788d445e60e4bc40891f2fee26044119a1")

In [11]:
commitsObj[0].sha

'20173f788d445e60e4bc40891f2fee26044119a1'

In [13]:
def getGhDir(org, repo, commit, path, dest, exclude=None):
  destPath = f'{dest}/{path}'
  if os.path.exists(destPath):
    rmtree(destPath)
  os.makedirs(destPath, exist_ok=True)
  
  g = Github()
  repoObj = g.get_repo(f'{org}/{repo}')
  
  excludeRe = re.compile(exclude) if exclude else None
  
  good = True
  
  def _getGhDir(subPath, level=0):
    nonlocal good
    lead = '\t' * level
    contents = repoObj.get_dir_contents(subPath, ref=commit)
    for content in contents:
      thisPath = content.path
      sys.stdout.write(f'{lead}{thisPath}...')
      if exclude and excludeRe.search(thisPath):
        print('excluded')
        continue
      if content.type == 'dir':
        print('directory')
        os.makedirs(f'{dest}/{thisPath}', exist_ok=True)
        _getGhDir(thisPath, level + 1)
      else:
        try:
          fileContent = repoObj.get_git_blob(content.sha)
          fileData = base64.b64decode(fileContent.content)
          fileDest = f'{dest}/{thisPath}'
          with open(fileDest, 'wb') as fd:
            fd.write(fileData)
          print('downloaded')
        except (GithubException, IOError) as exc:
          print('error')
          print(f'{lead}{exc}')
          good = False

  _getGhDir(path, 0)
  
  if good:
    print('OK')
  else:
    print('Done. There were errors')
    
  return good

In [16]:
getGhDir(ORG, REPO, COMMIT, 'parallels/tf', DWN, exclude=r'\.tf')

parallels/tf/1.0.1...directory
	parallels/tf/1.0.1/sim.tf...excluded
parallels/tf/1.0.4...directory
	parallels/tf/1.0.4/sim.tf...excluded
OK


True